# В данной работе будет исследоваться:
1. Зависимость количества ДТП от ям\неровностей на дороге
2. ...


## Installing dependecies, importing some of them and setting global variables

In [2]:
%pip install -qU setuptools pandas numpy plotly rich tqdm geopandas geojson bs4 requests ipywidgets jupyterlab_widgets pydantic

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
from rich import print
from tqdm.notebook import tqdm
from pathlib import Path
# from google.colab import output
# output.enable_custom_widget_manager()

data_accidents_source_url = 'https://dtp-stat.ru/opendata'

## Downloading data
If you are already have data in `./data` folder - skip this section

### Download HTML table and save links to list
source link was added in import section:
```python
data_accidents_source_usr = 'https://dtp-stat.ru/opendata'
```

In [4]:
from bs4 import BeautifulSoup
import urllib.request

html_page = urllib.request.urlopen(data_accidents_source_url)
soup = BeautifulSoup(html_page, "html.parser")
download_links = []
for link in soup.findAll('a'):
    link_href = link.get('href')
    download_links.append(link_href) if '.geojson' in link_href else None
print(download_links)

[
    'https://cms.dtp-stat.ru/media/opendata/altaiskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/amurskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/arkhangelskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/astrakhanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/belgorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/brianskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/vladimirskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/volgogradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/vologodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/voronezhskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/evreiskaia-avtonomnaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/zabaikalskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/ivanovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/irkutskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kabardino-balkarskaia-respublika.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kaliningradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kaluzhskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kamchatskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/karachaevo-cherkesskaia-respublika.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kemerovskaia-oblast-kuzbass.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kirovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kostromskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/krasnodarskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/krasnoiarskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kurganskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kurskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/leningradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/lipetskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/magadanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/moskva.geojson',
    'https://cms.dtp-stat.ru/media/opendata/moskovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/murmanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/nenetskii-avtonomnyi-okrug.geojson',
    'https://cms.dtp-stat.ru/media/opendata/nizhegorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/novgorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/novosibirskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/omskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/orenburgskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/orlovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/penzenskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/permskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/primorskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/pskovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-adygeia-adygeia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-altai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-bashkortostan.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-buriatiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-dagestan.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-ingushetiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-kalmykiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-kareliia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-komi.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-krym.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-marii

### Download every file and store it `./data/*.geojson` file
NOTE: `geojson_files` will be changed in next cell

In [5]:
import requests
from pathlib import Path

geojson_files = []
Path.mkdir(Path('data'), exist_ok=True)

for file_link in tqdm(download_links):
    file_data = requests.get(file_link).content
    with open(file_link.replace('https://cms.dtp-stat.ru/media/opendata/', 'data/'), 'wb+') as file:
        geojson_files.append(file.name)
        file.write(file_data)


  0%|          | 0/85 [00:00<?, ?it/s]

## Loading data from file and convert it to `CarAccident` class

### Load data from file and store it `geojson_objects` list

In [12]:
import json
import geojson 

# for those who already have the data downloaded, so you can skip the previous cell
geojson_files = [str(file) for file in Path.cwd().glob('data/*.geojson')]
geojson_objects = []

for file in tqdm(geojson_files):
    with open(file, 'r', encoding='utf-8') as f:
            data = json.loads(f.read())
            geojson_objects.append(geojson.FeatureCollection(data['features']))
            #print(data['features'][0]['properties'])
print(geojson_objects[0].features[0]['properties'])

[
    'd:\\Work\\School X\\year4\\car_accidents\\data\\altaiskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\amurskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\arkhangelskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\astrakhanskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\belgorodskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\brianskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\chechenskaia-respublika.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\cheliabinskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\chukotskii-avtonomnyi-okrug.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\chuvashskaia-respublika-chuvashiia.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\evreiskaia-avtonomnaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\iamalo-nenetskii-avtonomnyi-okrug.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\iaroslavskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\irkutskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\ivanovskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kabardino-balkarskaia-respublika.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kaliningradskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kaluzhskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kamchatskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\karachaevo-cherkesskaia-respublika.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kemerovskaia-oblast-kuzbass.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\khabarovskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\khanty-mansiiskii-avtonomnyi-okrug-iugra.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kirovskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kostromskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\krasnodarskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\krasnoiarskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kurganskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\kurskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\leningradskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\lipetskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\magadanskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\moskovskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\moskva.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\murmanskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\nenetskii-avtonomnyi-okrug.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\nizhegorodskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\novgorodskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\novosibirskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\omskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\orenburgskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\orlovskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\penzenskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\permskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\primorskii-krai.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\pskovskaia-oblast.geojson',
    'd:\\Work\\School X\\year4\\car_accidents\\data\\respublika-adygeia-adygeia.geojson',
    'd:\\Work\\Scho

### Create a dataclasses for better data analysis
NOTE that dataclasses are inherits from `pydantic.BaseModel`

In [9]:
from pydantic import BaseModel

class Participant(BaseModel):
    role: str | None
    gender: str | None
    violations: list | None
    health_status: str | None
    years_of_driving_experience: int | None

class Vehicle(BaseModel):
    year: int | None
    brand: str | None
    color: str | None
    category: str | None
    participants: list[Participant] | None

class CarAccident(BaseModel):
    id: int | None
    tags: list | None
    light: str | None
    point: dict | None
    nearby: list | None
    region: str | None
    scheme: str | None
    address: str | None
    weather: list | None 
    category: str | None
    datetime: str | None
    severity: str | None
    vehicles: list[Vehicle] | None 
    dead_count: int | None
    participants: list[Participant] | None
    injured_count: int | None
    parent_region: str | None 
    road_conditions: list | None
    participants_count: int | None
    participant_categories: list | None

In [10]:
# load properties from geojson_objects to CarAccident objects
car_accidents = []
for geojson_object in geojson_objects:
    for feature in geojson_object['features']:
        car_accident = CarAccident(**feature['properties'])
        car_accidents.append(car_accident)

print(car_accidents[0])

{'lat': 53.369632, 'long': 83.795021}

In [20]:
pickle.dump(car_accidents, open('car_accidents.pkl', 'wb'))

16

In [ ]:
car_accidents = pickle.load(open('car_accidents.pkl', 'rb'))
print(car_accidents[0])